In [ ]:
import xgboost as xgb
import pandas as pd

# Load training data
train_file = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/train_2010_2020.csv"
train_df = pd.read_csv(train_file)

# Load validation data
valid_file = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/valid_2021_2021.csv"
valid_df = pd.read_csv(valid_file)

print("Dataframe loaded!")

In [ ]:
# Take a look at the training set
print("Description of Training Data: \n", train_df.describe().to_markdown())
print("\nTraining Data Types\n", train_df.dtypes)


In [ ]:
# Take a look at the validation set
print("Description of Validation Data: \n", valid_df.describe().to_markdown())
print("\nValidation Data Types\n", valid_df.dtypes)

In [ ]:
# Split features and target
X_train = train_df.drop(columns=["Date", "Stock_ID", "Return_1d"])
y_train = train_df["Return_1d"]

X_valid = valid_df.drop(columns=["Date", "Stock_ID", "Return_1d"])
y_valid = valid_df["Return_1d"]

# Convert to DMatrix format
train_dmatrix = xgb.DMatrix(X_train, label=y_train)
valid_dmatrix = xgb.DMatrix(X_valid, label=y_valid)

# Set up hyperparameters
params = {
    'learning_rate': 0.01,  
    'max_depth': 24,  
    'min_child_weight': 1,  
    'gamma': 0,  
    'subsample': 0.8,  
    'colsample_bytree': 0.8,  
    'lambda': 0,  
    'alpha': 0,  
    'objective': 'reg:squarederror',  
    'booster': 'gbtree',  
    'eval_metric': 'rmse',  
    'tree_method': 'hist',  
    'device': 'cuda',  
}

# Train the model with validation
model = xgb.train(
    params=params, 
    dtrain=train_dmatrix, 
    num_boost_round=1000,  # Max trees
    evals=[(train_dmatrix, 'train'), (valid_dmatrix, 'valid')],  # Track validation
    early_stopping_rounds=50,  # Stops if no improvement in certain rounds
    verbose_eval=10  # Print every 10 rounds
)

# Save the trained model
model_path = '/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/xgboost_model.bin'
model.save_model(model_path)

print(f"Model trained successfully and saved at {model_path}")


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

preds = model.predict(xgb.DMatrix(X_valid))
mae = mean_absolute_error(y_valid, preds)
print("Validation MAE:", mae)

r2 = r2_score(y_valid, preds)
print("Validation R²:", r2)

xgb.plot_importance(model)
plt.show()


In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, confusion_matrix

# Load test data
test_path = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/training/test_2022_2025.csv"
test_df = pd.read_csv(test_path)

# Prepare X (features) and y (target: 1-day return)
X_test = test_df.drop(columns=["Date", "Stock_ID", "Return_1d"])  # Drop non-feature columns
y_test = test_df["Return_1d"]

# Convert test features to DMatrix
test_dmatrix = xgb.DMatrix(X_test)

# Load trained model
model_path = "/home/jesse/Projects/CWP_RL/03_XGBoost_Return_Prediction/xgboost_model.bin"
model = xgb.Booster()
model.load_model(model_path)

# Make predictions
y_pred = model.predict(test_dmatrix)

# Compute evaluation metrics
mse = mean_squared_error(y_test, y_pred)  # MSE
rmse = np.sqrt(mse)  # RMSE
mae = mean_absolute_error(y_test, y_pred)  # MAE
r2 = r2_score(y_test, y_pred)  # R² Score

# Convert predictions to binary (0 = negative, 1 = positive) for classification metrics
y_test_binary = (y_test > 0).astype(int)
y_pred_binary = (y_pred > 0).astype(int)

# Compute accuracy, F1 score, and confusion matrix
accuracy = accuracy_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary)
conf_matrix = confusion_matrix(y_test_binary, y_pred_binary)

# Organize results in a table (Pandas DataFrame)
metrics = {
    'Metric': ['RMSE', 'MAE', 'R² Score', 'Accuracy', 'F1 Score'],
    'Value': [rmse, mae, r2, accuracy, f1]
}

# Create DataFrame for metrics
metrics_df = pd.DataFrame(metrics)

# Confusion Matrix as DataFrame for better readability
conf_matrix_df = pd.DataFrame(conf_matrix, columns=['Predicted Negative', 'Predicted Positive'], 
                               index=['Actual Negative', 'Actual Positive'])

# Print the results
print("### Evaluation Metrics ###")
print(metrics_df)
print("\n### Confusion Matrix ###")
print(conf_matrix_df)
